# Symmetrix Extension irrep

This notebooks demonstrate algorithm in the paper

Tapping into Permutation Symmetry for Improved Detection of k-Symmetric Extensions [doi-link](https://doi.org/10.3390/e25101425)

TODO

1. how to write these basis
2. how to find these irrep basis

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt

import numqi

## Example: qutrits

For systems $B_1,B_2,\cdots,B_k$ each of dimension $d$, if the density matirx $\rho\in(\mathcal{H}^d)^{\otimes k}$ is symmetric with respect to the permutation group $S_k$, which means $\rho$ is invariant under any permutation of the subsystems:

$$ P_{B_iB_j} \rho P_{B_iB_j}=\rho $$

above, $P_{B_iB_j}$ is the permutation operator that swaps the $i$-th and $j$-th subsystems.

Then, the density matrix can be written as a direct sum of irreducible representations of the permutation group (TODO-formula):

$$ \rho = \oplus_\lambda \rho_{\lambda} $$

### 2 qutrits

2 qutrits can be divided into Bosonic basic (dimension of 6) and Fermionic basic (dimension of 3).

In [ ]:
dim = 3
rho = numqi.random.rand_ABk_density_matrix(dimA=1, dimB=dim, kext=2)
print('density matrix shape:', rho.shape)

Let's verify it's symmetric under permutation group $S_2$.

In [ ]:
rho_perm = rho.reshape(3, 3, 3, 3).transpose(1,0,3,2).reshape(9,9)
print('error:', np.abs(rho-rho_perm).max())

Then, let's put the density matrix into the irrep basis.

In [ ]:
basis_list = numqi.group.symext.get_sud_symmetric_irrep_basis(dim, kext=2)
basis_list = [y for x in basis_list for y in x] #different young tableaux
for ind0,basis in enumerate(basis_list):
    print(f'irrep-basis({ind0}), shape={basis.shape}')

# all irrep-basis make a complete basis (unitary matrix)
unitary = np.concatenate(basis_list, axis=0)
assert np.abs(unitary @ unitary.T.conj() - np.eye(9)).max() < 1e-10

rho_irrep = unitary.conj() @ rho @ unitary.T
rho_irrep[np.abs(rho_irrep)<1e-13] = 0 #remove items with abs value less than 1e-10
fig,ax = plt.subplots()
_ = ax.spy(np.abs(rho_irrep))

TODO-3 qutrits

## Symmetric extension with irrep

((TODO-formula))

In [ ]:
coeff_list,multiplicity_list = numqi.group.symext.get_symmetric_extension_irrep_coeff(dim=3, kext=2)
for ind0,coeff in enumerate(coeff_list):
    tmp0 = len(np.nonzero(np.abs(coeff)>1e-10)[0])
    print(f'irrep({ind0}), shape={coeff.shape}, #nonzero={tmp0}')
    numqi.group.symext.print_symmetric_extension_irrep_coeff(coeff)
    print()


## Time benchmark

Werner state boundary $\frac{k+d^2-d}{kd^2+d-1}$, the absolute error is within $10^{-8}$

In [ ]:
# the solver used to build github-pages is SCS, not MOSEK (not free)
case_dict = {
    2: [6,8,10,16],
    3: [2,3],
}

for dim,kext_list in case_dict.items():
    dm_werner = numqi.state.Werner(dim, alpha=1)
    dm_norm = numqi.gellmann.dm_to_gellmann_norm(dm_werner)
    for kext in kext_list:
        _ = numqi.group.symext.get_symmetric_extension_irrep_coeff(dim, kext) #build cache
        t0 = time.time()
        beta = numqi.entangle.symext.get_ABk_symmetric_extension_boundary(dm_werner, (dim,dim), kext, use_boson=False)
        alpha_irrep = (beta/dm_norm)*dim/(beta/dm_norm+dim-1)
        alpha_analytical = (kext+dim*dim-dim)/(kext*dim+dim-1)
        tmp0 = time.time() - t0
        print(f'[d={dim},kext={kext}][{tmp0:.3f}s] alpha={alpha_irrep:.6f}, abs(error)={abs(alpha_analytical-alpha_irrep):.5g}')


| $(d,k)$| QETLAB time (s)| irrep time (s) | $\alpha$ |
| :-: | :-: | :-: | :-: |
| $(2,6)$ | 0.14 | 0.10 | 0.615 |
| $(2,8)$ | 0.19 | 0.16 | 0.588 |
| $(2,10)$ | 12.60 | 0.16 | 0.571 |
| $(2,16)$ | NA | 0.32 | 0.545 |
| $(2,32)$ | NA | 3.18 | 0.523 |
| $(2,64)$ | NA | 51.96 | 0.512 |
| $(3,3)$ | 0.62 | 0.51 | 0.818 |
| $(3,4)$ | 7.96 | 2.38 | 0.714 |
| $(3,5)$ | NA | 11.56 | 0.647 |
| $(3,6)$ | NA | 55.60 | 0.6 |

1. computer spec
   * AMD R7-5800H, 16 cpu (hyperthreaded), 16GB memory
   * momery is the bottleneck, time may be slightly overestimated
2. QETLAB time: kext SDP from QETLAB
    * cvx solver for MATLAB
    * the matlab script to record time is stored in `project/ws_irrep/demo_qetlab_time.m`
3. irrep time: SDP with irreducible representation [doi-link](https://doi.org/10.3390/e25101425)
    * python/cvxpy with MOSEK solver, should be comparable to cvx


## Counting parameters

In [ ]:
dim = 3
kext_list = [3] #[3,4,5,6] #too time-consuming for github-pages
for kext in kext_list:
    coeff_list,multiplicity_list = numqi.group.symext.get_symmetric_extension_irrep_coeff(dim, kext) #build cache
    dim_list = [x.shape[0] for x in coeff_list]
    assert sum(x*y for x,y in zip(dim_list,multiplicity_list))==dim**kext
    tmp0 = ' + '.join([f'{x}x{y}' for x,y in zip(dim_list,multiplicity_list)])
    num_parameter = sum(x*x for x in dim_list)
    print(f'{dim}^{kext} = {tmp0}, #parameter={num_parameter}/{dim**(2*kext)}')


| $(d,k)$ | Young Tableau $d^k=\sum_i m_i\times s_i$ | parameters $(\sum_i s_i^2,d^{2k})$ |
| :-: | :-: | :-: |
| $(3,3)$ | $3^3 = 10\times 1 + 8\times 2 + 1\times 1$ | $(165,729)$ |
| $(3,4)$ | $3^4 = 15\times 1 + 15\times 3 + 6\times 2 + 3\times 3$ | $(495,6561)$ |
| $(3,5)$ | $3^5 = 21\times 1 + 24\times 4 + 15\times 5 + 6\times 6 + 3\times 5$ | $(1287,59049)$ |
| $(3,6)$ | $3^6 = 28\times 1 + 35\times 5 + 27\times 9 + 10\times 5 + 10\times 10 + 8\times 16 + 1\times 5$ | $(3003,531441)$ |
| $(4,3)$ | $4^3 = 20\times 1 + 20\times 2 + 4\times 1$ | $(816,4096)$ |
| $(4,4)$ | $4^4 = 35\times 1 + 45\times 3 + 20\times 2 + 15\times 3 + 1\times 1$ | $(3876,65536)$ |
| $(4,5)$ | $4^5 = 56\times 1 + 84\times 4 + 60\times 5 + 36\times 6 + 20\times 5 + 4\times 4$ | $(15504,1048576)$ |
